In [1]:
'''
all written function are here 

'''

'\nall written function are here \n\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical

In [3]:
# cleaning and pre processing data

dataPath = "data.csv"
data = pd.read_csv(dataPath)

# seperating features
label_column = 'Label' 
features = data.drop(columns=[label_column]).select_dtypes(include=['float64', 'int64'])
Labels = data[label_column]
# Add this to see excluded columns
excluded_columns = set(data.columns) - set(features.columns)
print("Excluded columns (including label):", excluded_columns)

#standardize
scaler = StandardScaler()
data_scaled = scaler.fit_transform(features)

Excluded columns (including label): {'SrcAddr', 'Dir', 'Attack Category', 'SrcMac', 'DstMac', 'Flgs', 'Sport', 'Label', 'DstAddr'}


In [4]:
def print_classification_results(confusion_matrix, classification_report, accuracy):
    #prints the cm results, classification report, and accuracy
    tn, fp, fn, tp = confusion_matrix.ravel()
    total = tn + fp + fn + tp
    
    # Calculate percentages
    tn_pct = (tn/total) * 100
    fp_pct = (fp/total) * 100
    fn_pct = (fn/total) * 100
    tp_pct = (tp/total) * 100
    
    # Print confusion matrix results
    print("\nConfusion Matrix Results:")
    print(f"""
    True Negatives: {tn} ({tn_pct:.2f}%)
    False Positives: {fp} ({fp_pct:.2f}%)
    False Negatives: {fn} ({fn_pct:.2f}%)
    True Positives: {tp} ({tp_pct:.2f}%)
    """)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report)
    
    # Print accuracy
    print(f"\nModel Accuracy: {accuracy:.2%}")


In [ ]:
def find_top_n_features(data_scaled, features, n):
    
    # -----------------applying PCA-----------------------
    pca = PCA(n_components=n)
    pca.fit(data_scaled)
    
    # Getting the top n features based on explained variance
    components = pca.components_
    #doesprint(components)
    explained_variance = pca.explained_variance_ratio_
    
    feature_importance = pd.DataFrame()
    for i in range(n):
        feature_importance[f'PC{i+1}'] = abs(components[i])
    
    feature_importance.index = features.columns
    
    # Getting overall importance by summing across components
    feature_importance['Overall_Importance'] = feature_importance.sum(axis=1)
    feature_importance = feature_importance.sort_values('Overall_Importance', ascending=False)

    print(f"Total explained variance ratio: {sum(explained_variance):.2%}")
    print("\nExplained variance ratio by component:")
    for i, var in enumerate(explained_variance):
        print(f"PC{i+1}: {var:.2%}")
    
    print(f"\nTop {n} most important features:")
    top_n_features = feature_importance['Overall_Importance'].head(n)
    print(top_n_features)
    
    # printing out the combinations
    print("\nPrincipal Component Linear Combinations:")
    
    for pc in range(3): #only want to print the top three
        print(f"\nPC{pc+1} = ", end="")
        # Get the components for this PC
        coefficients = components[pc]
        terms = []
        for feat, coef in zip(features.columns, coefficients):
            if abs(coef) > 0.1: #only print if higher than .1
                terms.append(f"({coef:.3f} × {feat})")
        print(" + ".join(terms))
    
    return feature_importance, top_n_features

In [25]:
find_top_n_features(data_scaled, features, 10)

Total explained variance ratio: 87.71%

Explained variance ratio by component:
PC1: 24.29%
PC2: 14.41%
PC3: 12.17%
PC4: 8.41%
PC5: 8.28%
PC6: 5.97%
PC7: 3.95%
PC8: 3.71%
PC9: 3.33%
PC10: 3.18%

Top 10 most important features:
Temp          1.598481
SrcBytes      1.539131
ST            1.538129
Pulse_Rate    1.483176
Heart_rate    1.481323
SYS           1.459602
sMinPktSz     1.453595
SIntPkt       1.451182
SrcLoad       1.441597
DIA           1.384256
Name: Overall_Importance, dtype: float64

Principal Component Linear Combinations:

PC1 = (0.141 × SrcBytes) + (0.258 × DstBytes) + (0.162 × SIntPkt) + (0.265 × DIntPkt) + (0.291 × SIntPktAct) + (0.293 × SrcJitter) + (0.278 × DstJitter) + (0.139 × sMaxPktSz) + (0.293 × Dur) + (0.246 × TotPkts) + (0.212 × TotBytes) + (0.305 × Loss) + (0.301 × pLoss) + (0.259 × pSrcLoss) + (0.288 × pDstLoss)

PC2 = (-0.272 × SrcBytes) + (-0.240 × DstBytes) + (0.210 × SrcLoad) + (0.313 × DstLoad) + (0.220 × SIntPkt) + (0.187 × SIntPktAct) + (0.185 × SrcJitte

(                     PC1           PC2           PC3           PC4  \
 Temp        9.752076e-04  8.023737e-03  4.697304e-03  4.237827e-02   
 SrcBytes    1.414744e-01  2.715990e-01  1.818744e-01  2.064207e-01   
 ST          2.909069e-03  3.490860e-03  1.590248e-02  1.491996e-01   
 Pulse_Rate  1.989962e-02  1.454387e-02  4.781668e-02  2.109341e-01   
 Heart_rate  3.795804e-03  8.790430e-03  3.214958e-02  2.356195e-01   
 SYS         7.325385e-04  2.827284e-03  2.286887e-02  2.413371e-01   
 sMinPktSz   4.950353e-02  2.206650e-01  2.842022e-01  2.969146e-01   
 SIntPkt     1.624856e-01  2.195511e-01  2.309146e-01  2.653060e-01   
 SrcLoad     9.953613e-02  2.098301e-01  1.901392e-01  3.476696e-01   
 DIA         3.022616e-03  1.838004e-03  1.514936e-02  1.679295e-01   
 Load        8.287298e-02  2.770905e-01  2.825988e-01  2.679442e-01   
 dMaxPktSz   7.533153e-04  1.414897e-02  8.438368e-04  1.808125e-02   
 sMaxPktSz   1.389667e-01  2.636301e-01  2.470389e-01  1.298151e-01   
 Dur  

In [7]:
def perform_kmeans_clustering(data_scaled, features, labels, n_components=2):
    """
    Perform K-means clustering on PCA components using selected features
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        features: DataFrame with only the features to analyze
        labels: target labels
        n_components: number of PCA components to use
    """
    print("\nperformin k means on this many features: ", len(features.columns))
    
    
    # Convert data_scaled to DataFrame with feature names
    scaled_df = pd.DataFrame(data_scaled, columns=features.columns)
    
    # Select only the specified features
    selected_features = scaled_df[features.columns]
    
    # PCA on selected features
    pca = PCA(n_components=n_components)
    pc_features = pca.fit_transform(selected_features)
    
    pca_df = pd.DataFrame(
        data=pc_features,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        pca_df, labels, test_size=0.1, random_state=42
    )
    
    # K-means clustering
    kmeans = KMeans(n_clusters=2, random_state=42)
    kmeans.fit(X_train)
    y_pred = kmeans.predict(X_test)
    
    # Evaluation metrics
    cm = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)    
    accuracy = accuracy_score(y_test, y_pred)    
    return cm, class_report, accuracy

In [8]:
cm, class_report, accuracy = perform_kmeans_clustering(data_scaled, features, data[label_column], n_components=3)

print_classification_results(cm, class_report, accuracy)


performin k means on this many features:  36

Confusion Matrix Results:

    True Negatives: 1436 (87.99%)
    False Positives: 2 (0.12%)
    False Negatives: 194 (11.89%)
    True Positives: 0 (0.00%)
    

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1438
           1       0.00      0.00      0.00       194

    accuracy                           0.88      1632
   macro avg       0.44      0.50      0.47      1632
weighted avg       0.78      0.88      0.82      1632


Model Accuracy: 87.99%


In [9]:
# feature_importance, top_features = find_top_n_features(data_scaled, features, 10)


In [10]:
def train_random_forest(data_scaled, Labels, features, n_estimators):
    """
    Train Random Forest on selected features
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        Labels: target labels 
        features: DataFrame with features to analyze
        n_estimators: number of trees in forest
    Returns:
        tuple: (confusion_matrix, classification_report, accuracy)
    """
    print(f"Training Random Forest with {n_estimators} estimators")
    print(f"Number of features used: {len(features.columns)}")
    
    # Convert scaled data to DataFrame with feature names
    scaled_df = pd.DataFrame(data_scaled, columns=features.columns)
    
    # Select only specified features
    selected_features = scaled_df[features.columns]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        selected_features, Labels, test_size=0.3, random_state=42
    )

    # Train model
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = rf_classifier.predict(X_test)

    # Calculate metrics
    cm = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    # Print feature importance
    feature_importance = pd.DataFrame({
        'feature': features.columns,
        'importance': rf_classifier.feature_importances_
    })
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    print("\nTop 5 Most Important Features:")
    print(feature_importance.head(5))
    
    return cm, class_report, accuracy

In [11]:
forest_cm, forest_report, forest_accuracy = train_random_forest(data_scaled, Labels, features, 100)

print_classification_results(forest_cm, forest_report, forest_accuracy)

Training Random Forest with 100 estimators
Number of features used: 36

Top 5 Most Important Features:
      feature  importance
8     DIntPkt    0.133990
21       Load    0.096434
12  DstJitter    0.093912
17        Dur    0.086107
3     SrcLoad    0.082480

Confusion Matrix Results:

    True Negatives: 4241 (86.62%)
    False Positives: 11 (0.22%)
    False Negatives: 313 (6.39%)
    True Positives: 331 (6.76%)
    

Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4252
           1       0.97      0.51      0.67       644

    accuracy                           0.93      4896
   macro avg       0.95      0.76      0.82      4896
weighted avg       0.94      0.93      0.92      4896


Model Accuracy: 93.38%


In [15]:
def perform_lda(data_scaled, labels, features):
    """
    Perform Linear Discriminant Analysis on selected features
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        labels: target labels
        features: DataFrame with features to analyze
    Returns:
        tuple: (confusion_matrix, classification_report, accuracy)
    """
    print(f"\nPerforming LDA on {len(features.columns)} features")
    
    # Convert scaled data to DataFrame with feature names
    scaled_df = pd.DataFrame(data_scaled, columns=features.columns)
    
    # Select only specified features
    selected_features = scaled_df[features.columns]
    
    # Split using the features DataFrame
    X_train, X_test, y_train, y_test = train_test_split(
        selected_features, labels, test_size=0.3, random_state=42
    )

    # Initialize and fit LDA model
    lda = LDA()
    lda.fit(X_train, y_train)

    # Print LDA components information
    print("\nLDA Components Information:")
    print(f"Number of components: {lda.n_components}")
    print("\nExplained variance ratio:")
    print(lda.explained_variance_ratio_)
    
    # Print component coefficients
    print("\nLinear Discriminant Coefficients:")
    for i, component in enumerate(lda.coef_):
        print(f"\nLD{i+1} coefficients:")
        for feat, coef in zip(features.columns, component):
            print(f"{feat}: {coef:.4f}")

    # Make predictions and calculate metrics
    y_pred = lda.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    return conf_matrix, class_report, accuracy

In [16]:
cm, report, accuracy = perform_lda(data_scaled, Labels, features)

print_classification_results(cm, report, accuracy)


Performing LDA on 36 features

LDA Components Information:
Number of components: None

Explained variance ratio:
[1.]

Linear Discriminant Coefficients:

LD1 coefficients:
Dport: 0.0000
SrcBytes: 4.5746
DstBytes: -9.3746
SrcLoad: -0.4146
DstLoad: -3.5580
SrcGap: -0.0000
DstGap: 0.0000
SIntPkt: -2.3615
DIntPkt: -0.4477
SIntPktAct: 453.4666
DIntPktAct: -0.0000
SrcJitter: -153.6353
DstJitter: 7.8913
sMaxPktSz: 0.0185
dMaxPktSz: -0.9048
sMinPktSz: -0.7674
dMinPktSz: 0.0000
Dur: 2.0393
Trans: 0.0000
TotPkts: 2.6564
TotBytes: -1.0525
Load: -2.3268
Loss: -552.0637
pLoss: 1173.2676
pSrcLoss: -384.6796
pDstLoss: -600.8920
Rate: 4.7021
Packet_num: -0.0375
Temp: -0.0457
SpO2: 0.0027
Pulse_Rate: 0.2497
SYS: -0.0287
DIA: 0.1639
Heart_rate: 0.0998
Resp_Rate: 0.0286
ST: 0.2382

Confusion Matrix Results:

    True Negatives: 4233 (86.46%)
    False Positives: 19 (0.39%)
    False Negatives: 351 (7.17%)
    True Positives: 293 (5.98%)
    

Classification Report:
              precision    recall  f1-

In [17]:
def gaussian_naive_bayes(data_scaled, labels, features):
    """
    Perform Gaussian Naive Bayes on selected features
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        labels: target labels
        features: DataFrame with features to analyze
    Returns:
        tuple: (confusion_matrix, classification_report, accuracy)
    """
    print(f"\nPerforming Gaussian Naive Bayes on {len(features.columns)} features")
    
    # Convert scaled data to DataFrame with feature names
    scaled_df = pd.DataFrame(data_scaled, columns=features.columns)
    
    # Select only specified features
    selected_features = scaled_df[features.columns]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        selected_features, labels, test_size=0.3, random_state=42
    )
    
    # Initialize and fit GNB model
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    
    # Make predictions and calculate metrics
    y_pred = gnb.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    return cm, class_report, accuracy

In [18]:
# Usage:
cm, report, acc = gaussian_naive_bayes(data_scaled, Labels, features)
print_classification_results(cm, report, acc)


Performing Gaussian Naive Bayes on 36 features

Confusion Matrix Results:

    True Negatives: 3843 (78.49%)
    False Positives: 409 (8.35%)
    False Negatives: 338 (6.90%)
    True Positives: 306 (6.25%)
    

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      4252
           1       0.43      0.48      0.45       644

    accuracy                           0.85      4896
   macro avg       0.67      0.69      0.68      4896
weighted avg       0.85      0.85      0.85      4896


Model Accuracy: 84.74%


In [19]:
def train_SVC(data_scaled, features, labels):
    """
    Train SVC models with different kernels using selected features
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        features: DataFrame with features to analyze
        labels: target labels
    Returns:
        tuple: (confusion_matrix, classification_report, accuracy) for best kernel
    """
    print("\nPerforming SVC on this many features: ", len(features.columns))
    
    # Convert data_scaled to DataFrame with feature names
    scaled_df = pd.DataFrame(data_scaled, columns=features.columns)
    
    # Select only specified features
    selected_features = scaled_df[features.columns]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        selected_features, labels, test_size=0.3, random_state=42
    )

    kernelTypes = ['linear', 'rbf', 'poly']
    best_accuracy = 0
    best_metrics = None
    
    print("\nTraining SVC models with different kernels...")
    for kernelType in tqdm(kernelTypes, desc='Training'):
        model = SVC(kernel=kernelType, C=1)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        
        print(f"\nKernel: {kernelType}")
        print(f"Accuracy: {accuracy:.2f}")
        
    return cm, report, accuracy
    



In [20]:
# Usage:
cm, report, acc = train_SVC(data_scaled, features, Labels)
print_classification_results(cm, report, acc)


Performing SVC on this many features:  36

Training SVC models with different kernels...


Training:   0%|          | 0/3 [00:00<?, ?it/s]

Training:  33%|███▎      | 1/3 [00:06<00:12,  6.40s/it]


Kernel: linear
Accuracy: 0.93


Training:  67%|██████▋   | 2/3 [00:08<00:03,  3.71s/it]


Kernel: rbf
Accuracy: 0.93


Training: 100%|██████████| 3/3 [00:09<00:00,  3.16s/it]


Kernel: poly
Accuracy: 0.93

Confusion Matrix Results:

    True Negatives: 4244 (86.68%)
    False Positives: 8 (0.16%)
    False Negatives: 342 (6.99%)
    True Positives: 302 (6.17%)
    

Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4252
           1       0.97      0.47      0.63       644

    accuracy                           0.93      4896
   macro avg       0.95      0.73      0.80      4896
weighted avg       0.93      0.93      0.92      4896


Model Accuracy: 92.85%


In [21]:
def CNN(data_scaled, labels, features):
    """
    Perform CNN classification on selected features
    
    Args:
        data_scaled: scaled numpy array from StandardScaler
        labels: target labels
        features: DataFrame with features to analyze
    Returns:
        tuple: (confusion_matrix, classification_report, accuracy)
    """
    print(f"\nPerforming CNN on {len(features.columns)} features")
    
    # Convert scaled data to DataFrame with feature names
    scaled_df = pd.DataFrame(data_scaled, columns=features.columns)
    
    # Select only specified features
    selected_features = scaled_df[features.columns].values
    
    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(labels)
    y_categorical = to_categorical(y_encoded)

    # Reshape data for CNN
    num_features = selected_features.shape[1]
    grid_size = int(np.ceil(np.sqrt(num_features)))
    X_padded = np.zeros((selected_features.shape[0], grid_size * grid_size))
    X_padded[:, :num_features] = selected_features
    X_reshaped = X_padded.reshape(-1, grid_size, grid_size, 1)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_reshaped, y_categorical, test_size=0.3, random_state=42
    )

    # Build CNN model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(grid_size, grid_size, 1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, kernel_size=(2, 2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(y_categorical.shape[1], activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train model
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, 
                       validation_data=(X_test, y_test), verbose=0)

    # Get predictions
    y_pred_proba = model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)

    # Calculate metrics
    cm = confusion_matrix(y_test_labels, y_pred)
    class_report = classification_report(y_test_labels, y_pred)
    accuracy = accuracy_score(y_test_labels, y_pred)
    
    return cm, class_report, accuracy

In [22]:
# Run the CNN model
cnn_cm, cnn_report, cnn_accuracy = CNN(data_scaled, Labels, features)

# Print the results
print_classification_results(cnn_cm, cnn_report, cnn_accuracy)


Performing CNN on 36 features


c:\Users\allen\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Confusion Matrix Results:

    True Negatives: 4225 (86.29%)
    False Positives: 27 (0.55%)
    False Negatives: 299 (6.11%)
    True Positives: 345 (7.05%)
    

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4252
           1       0.93      0.54      0.68       644

    accuracy                           0.93      4896
   macro avg       0.93      0.76      0.82      4896
weighted avg       0.93      0.93      0.93      4896


Model Accuracy: 93.34%
